In [17]:
def read_graph_from_edges(file_path):
    # Initialize an empty dictionary to store the graph
    graph = {}
    
    # Open the file and read it line by line
    with open(file_path, 'r') as file:
        for line in file:
            # Skip comment lines (lines starting with %)
            if line.startswith('%'):
                continue
            
            # Read the source and target nodes of the edge
            source, target = map(int, line.strip().split())
            
            # Build adjacency list
            if source not in graph:
                graph[source] = []
            if target not in graph[source]:  # Avoid duplicate edges
                graph[source].append(target)
            
            # If the graph is undirected, add the reverse edge
            if target not in graph:
                graph[target] = []
            if source not in graph[target]:  # Avoid duplicate edges
                graph[target].append(source)
    
    # Return the constructed graph
    return graph

# Example usage
file_path = 'brunson_corporate-leadership/out.brunson_corporate-leadership_corporate-leadership'  
graph = read_graph_from_edges(file_path)


In [15]:
import random

# Generate a random subgraph
def generate_random_subgraph(graph, p):
    """
    Generate a random subgraph based on propagation probability p.

    Parameters:
    - graph: The graph represented as an adjacency list.
    - p: Propagation probability.

    Returns:
    - subgraph: A new graph where edges are kept with probability p.
    """
    subgraph = {node: [] for node in graph}
    for source, neighbors in graph.items():
        for target in neighbors:
            if random.random() <= p:  # Keep the edge with probability p
                subgraph[source].append(target)
    return subgraph

# Calculate the reachable nodes starting from a set of seed nodes (connected component)
def compute_reachable_nodes(graph, seed_set):
    """
    Use BFS to compute the set of nodes reachable from a set of seed nodes.

    Parameters:
    - graph: The graph represented as an adjacency list.
    - seed_set: A set of seed nodes.

    Returns:
    - visited: A set of all reachable nodes.
    """
    visited = set()
    queue = list(seed_set)
    
    while queue:
        current = queue.pop(0)
        if current not in visited:
            visited.add(current)
            queue.extend(graph[current])  # Add all neighbors of the current node
    
    return visited

# Implementation of the NewGreedyIC algorithm
def new_greedy_ic(graph, k, p, R=20000):
    """
    Implement the NewGreedyIC algorithm to select seed nodes.

    Parameters:
    - graph: The graph represented as an adjacency list.
    - k: Number of seed nodes to select.
    - p: Propagation probability.
    - R: Number of simulations for each round.

    Returns:
    - seed_set: The set of selected seed nodes.
    """
    seed_set = set()  # Initialize the seed node set
    nodes = list(graph.keys())  # List of all nodes
    
    for _ in range(k):  # Select k seed nodes
        max_spread = -1
        best_node = None
        
        # Iterate over all unselected nodes
        for candidate in nodes:
            if candidate in seed_set:
                continue
            
            # Simulate R times to estimate the spread of the candidate node
            total_spread = 0
            for _ in range(R):
                # Generate a random subgraph
                subgraph = generate_random_subgraph(graph, p)
                
                # Calculate the spread of the current seed set with the candidate node
                reachable = compute_reachable_nodes(subgraph, seed_set | {candidate})
                total_spread += len(reachable)
            
            # Compute the average spread
            avg_spread = total_spread / R
            
            # Update the best node
            if avg_spread > max_spread:
                max_spread = avg_spread
                best_node = candidate
        
        # Add the best node to the seed set
        seed_set.add(best_node)
        print(f"Selected node: {best_node}, Spread: {max_spread}")
    
    return seed_set

# Example usage
file_path = 'brunson_corporate-leadership/out.brunson_corporate-leadership_corporate-leadership' 
graph = read_graph_from_edges(file_path)

# Call the NewGreedyIC algorithm
k = 5  # Select 5 seed nodes
p = 0.1  # Propagation probability of 0.1
seed_set = new_greedy_ic(graph, k, p)
print("Final seed set:", seed_set)


Selected node: 5, Spread: 4.0849
Selected node: 4, Spread: 6.46405
Selected node: 18, Spread: 8.1899
Selected node: 2, Spread: 9.7125
Selected node: 17, Spread: 10.9511
Final seed set: {2, 4, 5, 17, 18}


In [16]:
import heapq
import random

def influence_spread(graph, seeds, p):
    """
    Simulate influence propagation and calculate the spread.

    Parameters:
    - graph: dict, adjacency list representation of the graph.
    - seeds: list, initial seed nodes.
    - p: float, propagation probability.

    Returns:
    - int: Total number of activated nodes.
    """
    active_nodes = set(seeds)  # Set of nodes already activated.
    newly_active = set(seeds)  # Nodes activated in the current step.

    while newly_active:
        next_newly_active = set()
        for node in newly_active:
            for neighbor in graph[node]:
                # Activate the neighbor with probability p if it is not already active
                if neighbor not in active_nodes and random.random() < p:
                    active_nodes.add(neighbor)
                    next_newly_active.add(neighbor)
        newly_active = next_newly_active  # Update for the next round

    return len(active_nodes)

def celf(graph, k, p, iterations=1000):
    """
    CELF (Cost-Effective Lazy Forward) algorithm implementation for seed selection.

    Parameters:
    - graph: dict, adjacency list representation of the graph.
    - k: int, number of seed nodes to select.
    - p: float, propagation probability.
    - iterations: int, number of simulations for estimating influence spread.

    Returns:
    - list: Selected seed nodes.
    """
    # Initialization
    S = []  # Seed set
    marginal_gain = {}  # Marginal gain for each node
    Q = []  # Priority queue

    # Calculate initial marginal gains
    for node in graph:
        # Estimate the influence spread for each node individually
        gain = sum(influence_spread(graph, [node], p) for _ in range(iterations)) / iterations
        marginal_gain[node] = gain
        # Use negative gain for max-heap behavior in Python's min-heap
        heapq.heappush(Q, (-gain, node))

    # Select seed nodes
    for _ in range(k):
        while True:
            gain, node = heapq.heappop(Q)  # Extract the node with the highest marginal gain
            gain = -gain  # Convert back to positive value
            # If the node's gain is still valid, add it to the seed set
            if marginal_gain[node] == gain:
                S.append(node)
                # Update marginal gains for remaining nodes
                for neighbor in graph:
                    if neighbor not in S:
                        new_gain = sum(influence_spread(graph, S + [neighbor], p) for _ in range(iterations)) / iterations
                        marginal_gain[neighbor] = new_gain
                        heapq.heappush(Q, (-new_gain, neighbor))
                break

    return S

file_path = 'brunson_corporate-leadership/out.brunson_corporate-leadership_corporate-leadership'
graph = read_graph_from_edges(file_path)

# Parameters
k = 5  # Number of seed nodes to select
p = 0.01  # Propagation probability

# Run CELF algorithm
selected_seeds = celf(graph, k, p, iterations=1000)
print("Selected seed nodes:", selected_seeds)


Selected seed nodes: [6, 5, 4, 18, 8]


In [ ]:
def degree_discount_ic(graph, k, p):
    """
    Select seed nodes using the degree discount heuristic algorithm to maximize influence spread.

    Parameters:
    - graph: dict, representing the graph as an adjacency list. 
             Keys are nodes, and values are lists of neighbors.
    - k: int, the number of seed nodes to select.
    - p: float, propagation probability.

    Returns:
    - seeds: list, containing the selected seed nodes.
    """
    # Initialize
    degree = {v: len(neighbors) for v, neighbors in graph.items()}  # Degree of each node
    t = {v: 0 for v in graph}  # Number of neighbors already selected as seeds for each node
    dd = degree.copy()  # Initialize degree discount values, a dictionary where the keys are nodes, and the values are the degree discount values for those nodes.
    seeds = []  # List to store selected seed nodes
    
    for _ in range(k):
        # Select the node with the maximum degree discount value as the new seed
        u = max(dd, key=dd.get)
        seeds.append(u)
        
        # Update the degree discount values of the neighbors of the newly selected seed
        for v in graph[u]:
            if v not in seeds:  # If the neighbor has not been selected as a seed
                t[v] += 1  # Increment the count of seed neighbors for this node
                dd[v] = degree[v] - 2 * t[v] - (degree[v] - t[v]) * t[v] * p  # Update degree discount value
        
        # Remove the degree discount value of the node already selected as a seed
        dd.pop(u)
    
    return seeds


# Parameters
k = 5  # Select 5 seed nodes
p = 0.01  # Propagation probability

# Run the algorithm
file_path = 'brunson_corporate-leadership/out.brunson_corporate-leadership_corporate-leadership' 
graph = read_graph_from_edges(file_path)
seeds = degree_discount_ic(graph, k, p)
print("Selected seed nodes:", seeds)


Selected seed nodes: [5, 4, 18, 6, 8]
